In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [14]:
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
input_width=28
input_height=28
input_channels=1
input_pixels=784

stride_conv1=1
stride_conv2=1
n_conv1=32
n_conv2=64 #no. of layers
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10

input_size_to_hidden=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2

In [0]:
weights={
    "wc1":tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2":tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1":tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo":tf.Variable(tf.random_normal([n_hidden,n_out]))

}

biases={
    
    "bc1":tf.Variable(tf.random_normal([n_conv1])),
    "bc2":tf.Variable(tf.random_normal([n_conv2])),
    "bh1":tf.Variable(tf.random_normal([n_hidden])),
    "bo":tf.Variable(tf.random_normal([n_out]))
}


In [0]:
def conv(x,weights,bias,strides=1):
  out=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,strides,strides,1])
  out=tf.nn.bias_add(out,bias)
  out=tf.nn.relu(out)
  return out

def maxpooling(x,k=2):
  return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

  

In [0]:
#forward propagation
def cnn(x,weights,biases):
  x=tf.reshape(x,shape=[-1,input_height,input_width,input_channels])
  conv1=conv(x,weights["wc1"],biases["bc1"],stride_conv1)
  conv1_pool=maxpooling(conv1,max_pool1_k)
  
  conv2=conv(conv1_pool,weights["wc2"],biases["bc2"],stride_conv2)
  conv2_pool=maxpooling(conv2,max_pool2_k)
  
  hidden_input=tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])
  hidden_output_before_activation=tf.add(tf.matmul(hidden_input,weights["wh1"]),biases["bh1"])
  hidden_output=tf.nn.relu(hidden_output_before_activation)
  
  output=tf.add(tf.matmul(hidden_output,weights["wo"]),biases["bo"])
  
  return output

In [0]:
x=tf.placeholder("float",[None,input_pixels])
y=tf.placeholder(tf.int32,[None,n_out])
pred=cnn(x,weights,biases)

In [0]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))


In [0]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [0]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
batch_size=100
for i in range(25):
  num_batches=int(mnist.train.num_examples/batch_size)
  total_cost=0
  for j in range(num_batches):
    batch_x,batch_y=mnist.train.next_batch(batch_size)
    c,_=sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y})
    total_cost+=c
    print(total_cost)
    

75003.9296875
139021.1015625
247563.640625
341692.84375
426249.234375
528102.8359375
585198.78125
624340.578125
658866.03125
681142.685546875
691504.5166015625
698164.0278320312
705242.7885742188
711319.0883789062
719204.1508789062
725967.6650390625
732944.8598632812
740781.2407226562
745325.5004882812
749335.0541992188
751709.8479003906
753817.6525878906
755675.3145751953
757501.1246337891
759464.4381103516
760904.4699707031
761832.1804199219
763207.5135498047
764138.8513793945
764613.6416625977
765411.2490844727
766147.2454223633
766694.1993408203
767544.3070068359
768812.3671875
769887.0043945312
770441.0751953125
771220.8555297852
771979.3145751953
772514.2128295898
773048.2170410156
773809.0452270508
774331.1829833984
775077.2099609375
775333.8590698242
775757.721282959
776306.9805603027
776546.6520690918
777053.2353515625
777363.3473510742
777847.9196166992
778297.8969726562
778840.1720581055
779329.9526062012
779741.8710021973
779970.8014678955
780351.7632598877
780634.624893188

In [0]:
predictions=tf.argmax(pred,1)
correct_labels=tf.argmax(y,1)
correct_predictions=tf.equal(predictions,correct_labels)
predictions,correct_preds=sess.run([predictions,correct_predictions],feed_dict={x:mnist.test.images,y:mnist.test.labels})
correct_preds.sum()